In [5]:
import os
import numpy as np
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import models,layers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder

In [6]:
def getDataSize(path):
  image_count = {}

  for x in os.listdir(path):
    image_count[x] = len(os.listdir(os.path.join(path, x)))

  return image_count

In [7]:
train_path = 'train'
test_path = 'test'
valid_path = 'valid'

In [8]:
batch_size = 8


In [9]:
import os
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to your train and test data directories
train_path = 'train'
test_path = 'test'

# Define data generators for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),  # MobileNetV2 expects input images with shape (224, 224, 3)
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),  # MobileNetV2 expects input images with shape (224, 224, 3)
    batch_size=32,
    class_mode='categorical'
)

# Create input tensor
inputs = Input(shape=(224, 224, 3))  # MobileNetV2 expects input images with shape (224, 224, 3)

# Create MobileNetV2 model with pre-trained weights
base_model = MobileNetV2(weights=None, include_top=False, input_tensor=inputs)

# Load the weights file locally
local_weights_file = 'mobilenetv2.h5'  # Path to your local weights file
base_model.load_weights(local_weights_file, by_name=True)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of MobileNetV2
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)

num_classes = len(train_data.class_indices)  # Number of classes in your dataset

# Modify the last dense layer to have the same number of units as the number of classes
preds = Dense(num_classes, activation='softmax')(x)

# Combine base model and custom layers into a new model
model = Model(inputs=base_model.input, outputs=preds)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_data,
    steps_per_epoch=len(train_data),
    epochs=20,
    validation_data=test_data,
    validation_steps=len(test_data)
)


Found 743 images belonging to 4 classes.


Found 127 images belonging to 4 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 4,160,708 (15.87 MB)

...

2024-04-01 20:38:15.546467: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-01 20:38:15.548601: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 211ms/step - accuracy: 0.9984 - loss: 0.0110 - val_accuracy: 0.8898 - val_loss: 0.5463
Epoch 18/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 19/20


2024-04-01 20:38:21.463078: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-01 20:38:21.465150: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 215ms/step - accuracy: 0.9917 - loss: 0.0189 - val_accuracy: 0.9134 - val_loss: 0.4716
Epoch 20/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


2024-04-01 20:38:27.468162: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-01 20:38:27.470382: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [10]:
print(train_path)

train


In [11]:
# Get the maximum validation accuracy achieved during training
max_val_accuracy = max(history.history['val_accuracy'])

# Print the maximum validation accuracy
print("Best Validation Accuracy:", max_val_accuracy)


Best Validation Accuracy: 0.913385808467865


In [12]:
test_image = 'valid/large_cell_carcinoma/000108.png'

In [13]:
from PIL import Image

# Load a single image for testing
test_image_path = 'test/normal/7.png'
test_img = Image.open(test_image_path)
test_img = test_img.resize((224, 224))  # Resize the image to (224, 224)
test_img_gray = test_img.convert('L')   # Convert the image to grayscale

# Convert the grayscale image to numpy array
test_img_array = np.array(test_img_gray)
test_img_array = np.expand_dims(test_img_array, axis=0)  # Add batch dimension

# Rescale pixel values to [0, 1]
test_img_array = test_img_array / 255.0

# Reshape the array to match the model's input shape
test_img_array = np.expand_dims(test_img_array, axis=-1)  # Add channel dimension

# Now `test_img_array` is ready to be fed to your trained model for prediction


In [14]:
from keras.preprocessing import image
import numpy as np

# Define the target image size
target_size = (224, 224)

# Load and preprocess the single image
test_image_path = 'test/adenocarcinoma/000122.png'


test_img = image.load_img(test_image_path, target_size=target_size)
test_img_array = image.img_to_array(test_img)
test_img_array = np.expand_dims(test_img_array, axis=0)
preprocessed_img = test_img_array / 255.0  # Normalize pixel values

# Now `preprocessed_img` is ready to be fed to your trained model for prediction
output_probabilities = model.predict(preprocessed_img)

# Get the predicted class index
predicted_class_index = np.argmax(output_probabilities)

print("Predicted Class Index:", predicted_class_index)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
Predicted Class Index: 0


In [15]:
from keras.models import load_model

# Save the entire model to a file
model.save('best_model1.keras')